In this project, we used [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) library to train a custom object detection model capable of detecting salads within images on a mobile device. 

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with own custom dataset reduces the amount of training data required and will shorten the training time.

We used the publicly available *Salads* dataset, which was created from the [Open Images Dataset V4](https://storage.googleapis.com/openimages/web/index.html).

Each image in the dataset contains objects labeled as one of the following classes: 
* Baked Good
* Cheese
* Salad
* Seafood
* Tomato

The dataset contains the bounding-boxes specifying where each object locates, together with the object's label. 

## Prerequisites


### Install the required packages

In [15]:
!pip install -q tflite-model-maker
!pip install -q pycocotools
!pip install -q tflite-support

Import the required packages.

In [3]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
#Downloading the dataset

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="unauthorized")
project = rf.workspace("gw8717navercom").project("salad-data")
dataset = project.version(1).download("tensorflow")

### Prepare the dataset

The *Salads* dataset is available at:
 `gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv.` 

It contains 175 images for training, 25 images for validation, and 25 images for testing. The dataset has five classes: `Salad`, `Seafood`, `Tomato`, `Baked goods`, `Cheese`.

<br/>

The dataset is provided in CSV format:
```
TRAINING,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Salad,0.0,0.0954,,,0.977,0.957,,
VALIDATION,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Seafood,0.0154,0.1538,,,1.0,0.802,,
TEST,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Tomato,0.0,0.655,,,0.231,0.839,,
```

* Each row corresponds to an object localized inside a larger image, with each object specifically designated as test, train, or validation data.
* The three lines included here indicate **three distinct objects located inside the same image** available at `gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg`.
* Each row has a different label: `Salad`, `Seafood`, `Tomato`, etc.
* Bounding boxes are specified for each image using the top left and bottom right vertices.

## Train your salad detection model

#### Object detection model archiecture.

We use the EfficientDet-Lite2 model. EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture. 

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Pixel 4 using 4 threads on CPU [Ref: Google TF Lite Documentaion]. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>


In [4]:
spec = model_spec.get('efficientdet_lite2')

**Step 2. Load the dataset.**

Model Maker will take input data in the CSV format. Use the `ObjectDetectorDataloader.from_csv` method to load the dataset and split them into the training, validation and test images.


You can load the CSV file directly from Google Cloud Storage, but you don't need to keep your images on Google Cloud to use Model Maker. We can specify a local CSV file on your computer, and Model Maker will work just fine.

In [5]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

**Step 3. Train the TensorFlow model with the training data.**

* The EfficientDet-Lite0 model uses `epochs = 50` by default.
* Set `batch_size = 8` here so we will see that it takes 21 steps to go through the 175 images in the training dataset. 
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [6]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
21/21 [==============================] - 79s 2s/step - det_loss: 1.7540 - cls_loss: 1.1316 - box_loss: 0.0124 - reg_l2_loss: 0.0764 - loss: 1.8305 - learning_rate: 0.0090 - gradient_norm: 0.7342 - val_det_loss: 1.6683 - val_cls_loss: 1.1064 - val_box_loss: 0.0112 - val_reg_l2_loss: 0.0764 - val_loss: 1.7447
Epoch 2/50
21/21 [==============================] - 28s 1s/step - det_loss: 1.5948 - cls_loss: 1.0771 - box_loss: 0.0104 - reg_l2_loss: 0.0764 - loss: 1.6713 - learning_rate: 0.0100 - gradient_norm: 1.0794 - val_det_loss: 1.4721 - val_cls_loss: 0.9567 - val_box_loss: 0.0103 - val_reg_l2_loss: 0.0764 - val_loss: 1.5485
Epoch 3/50
21/21 [==============================] - 32s 2s/step - det_loss: 1.3778 - cls_loss: 0.9399 - box_loss: 0.0088 - reg_l2_loss: 0.0764 - loss: 1.4542 - learning_rate: 0.0099 - gradient_norm: 1.9733 - val_det_loss: 1.2991 - val_cls_loss: 0.8070 - val_box_loss: 0.0098 - val_reg_l2_loss: 0.0764 - val_loss: 1.3755
Epoch 4/50
21/21 [======================

**Step 4. Evaluate the model with the test data.**

After training the object detection model using the images in the training dataset, we use the remaining 25 images in the test dataset to evaluate how the model performs against new data.

As the default batch size is 64, it will take 1 step to go through the 25 images in the test dataset.

In [7]:
model.evaluate(test_data)

1/1 [==============================] - 9s 9s/step



{'AP': 0.23125926,
 'AP50': 0.39211,
 'AP75': 0.22839592,
 'AP_/Baked Goods': 0.056009296,
 'AP_/Cheese': 0.25100884,
 'AP_/Salad': 0.59339285,
 'AP_/Seafood': 0.019524494,
 'AP_/Tomato': 0.23636083,
 'APl': 0.23417331,
 'APm': 0.2912734,
 'APs': -1.0,
 'ARl': 0.42843384,
 'ARm': 0.49166667,
 'ARmax1': 0.17963235,
 'ARmax10': 0.37321636,
 'ARmax100': 0.43112075,
 'ARs': -1.0}

**Step 5.  Export as a TensorFlow Lite model.**

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is full integer quantization.

In [8]:
model.export(export_dir='.')

**Step 6.  Evaluate the TensorFlow Lite model.**

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop. 
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [9]:
model.evaluate_tflite('model.tflite', test_data)

25/25 [==============================] - 191s 8s/step



{'AP': 0.19890182,
 'AP50': 0.34420288,
 'AP75': 0.20404717,
 'AP_/Baked Goods': 0.035757806,
 'AP_/Cheese': 0.20603977,
 'AP_/Salad': 0.536264,
 'AP_/Seafood': 0.0005638154,
 'AP_/Tomato': 0.2158837,
 'APl': 0.20219831,
 'APm': 0.28807756,
 'APs': -1.0,
 'ARl': 0.28367373,
 'ARm': 0.39166668,
 'ARmax1': 0.14983957,
 'ARmax10': 0.2678721,
 'ARmax100': 0.28562945,
 'ARs': -1.0}

In the final step , we used the [ObjectDetector API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) of the [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview) to integrate the model into the Android app.